In [34]:
import keras
from keras.preprocessing import image
import numpy as np
from joblib import load
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
model = keras.models.load_model("Trained_model") 
clf = load('trained_model_text.joblib')
tfidf_vectorizer = load('tfidf_vectorizer.joblib')
m_lbl_bin = load('m_lbl_bin.joblib')

In [3]:
posters_genres = ['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'History', 'Horror',
       'Music', 'Musical', 'Mystery', 'N/A', 'News', 'Reality-TV', 'Romance',
       'Sci-Fi', 'Short', 'Sport', 'Thriller', 'War', 'Western']

In [111]:
plot_text_genres = ['Absurdism', 'Acid western', 'Action', 'Action Comedy', 'Action Thrillers', 'Action/Adventure', 'Addiction Drama', 'Adult', 'Adventure', 'Adventure Comedy', 'Airplanes and airports', 'Albino bias', 'Alien Film', 'Alien invasion', 'Americana', 'Animal Picture', 'Animals', 'Animated Musical', 'Animated cartoon', 'Animation', 'Anime', 'Anthology', 'Anthropology', 'Anti-war', 'Anti-war film', 'Apocalyptic and post-apocalyptic fiction', 'Archaeology', 'Archives and records', 'Art film', 'Auto racing', 'Avant-garde', 'B-Western', 'B-movie', 'Backstage Musical', 'Baseball', 'Beach Film', 'Beach Party film', 'Bengali Cinema', 'Biker Film', 'Biographical film', 'Biography', 'Biopic [feature]', 'Black comedy', 'Black-and-white', 'Blaxploitation', 'Bloopers & Candid Camera', 'Bollywood', 'Boxing', 'Breakdance', 'British Empire Film', 'British New Wave', 'Bruceploitation', 'Buddy Picture', 'Buddy cop', 'Buddy film', 'Business', 'C-Movie', 'Camp', 'Caper story', 'Cavalry Film', 'Chase Movie', 'Chick flick', 'Childhood Drama', "Children's", "Children's Entertainment", "Children's Fantasy", "Children's Issues", "Children's/Family", 'Chinese Movies', 'Christian film', 'Christmas movie', 'Clay animation', 'Cold War', 'Combat Films', 'Comdedy', 'Comedy', 'Comedy Thriller', 'Comedy Western', 'Comedy film', 'Comedy horror', 'Comedy of Errors', 'Comedy of manners', 'Comedy-drama', 'Coming of age', 'Coming-of-age film', 'Computer Animation', 'Computers', 'Concert film', 'Conspiracy fiction', 'Costume Adventure', 'Costume Horror', 'Costume drama', 'Courtroom Comedy', 'Courtroom Drama', 'Creature Film', 'Crime', 'Crime Comedy', 'Crime Drama', 'Crime Fiction', 'Crime Thriller', 'Cult', 'Culture & Society', 'Cyberpunk', 'Czechoslovak New Wave', 'Dance', 'Demonic child', 'Detective', 'Detective fiction', 'Disaster', 'Docudrama', 'Documentary', 'Dogme 95', 'Domestic Comedy', 'Doomsday film', 'Drama', 'Dystopia', 'Ealing Comedies', 'Early Black Cinema', 'Education', 'Educational', 'Ensemble Film', 'Environmental Science', 'Epic', 'Epic Western', 'Erotic Drama', 'Erotic thriller', 'Erotica', 'Escape Film', 'Essay Film', 'Existentialism', 'Experimental film', 'Exploitation', 'Expressionism', 'Extreme Sports', 'Fairy tale', 'Family & Personal Relationships', 'Family Drama', 'Family Film', 'Family-Oriented Adventure', 'Fan film', 'Fantasy', 'Fantasy Adventure', 'Fantasy Comedy', 'Fantasy Drama', 'Feature film', 'Female buddy film', 'Feminist Film', 'Fictional film', 'Filipino', 'Filipino Movies', 'Film', 'Film & Television History', 'Film adaptation', 'Film noir', 'Film à clef', 'Film-Opera', 'Filmed Play', 'Finance & Investing', 'Foreign legion', 'Future noir', 'Gangster Film', 'Gay', 'Gay Interest', 'Gay Themed', 'Gay pornography', 'Gender Issues', 'Giallo', 'Glamorized Spy Film', 'Goat gland', 'Gothic Film', 'Graphic & Applied Arts', 'Gross out', 'Gross-out film', 'Gulf War', 'Hagiography', 'Hardcore pornography', 'Haunted House Film', 'Health & Fitness', 'Heaven-Can-Wait Fantasies', 'Heavenly Comedy', 'Heist', 'Hip hop movies', 'Historical Documentaries', 'Historical Epic', 'Historical drama', 'Historical fiction', 'History', 'Holiday Film', 'Homoeroticism', 'Horror', 'Horror Comedy', 'Horse racing', 'Humour', 'Hybrid Western', 'Illnesses & Disabilities', 'Indian Western', 'Indie', 'Inspirational Drama', 'Instrumental Music', 'Interpersonal Relationships', 'Inventions & Innovations', 'Japanese Movies', 'Journalism', 'Jukebox musical', 'Jungle Film', 'Juvenile Delinquency Film', 'Kafkaesque', 'Kitchen sink realism', 'LGBT', 'Language & Literature', 'Latino', 'Law & Crime', 'Legal drama', 'Libraries and librarians', 'Linguistics', 'Live action', 'Malayalam Cinema', 'Marriage Drama', 'Martial Arts Film', 'Master Criminal Films', 'Media Satire', 'Media Studies', 'Medical fiction', 'Melodrama', 'Mockumentary', 'Mondo film', 'Monster', 'Monster movie', 'Movie serial', 'Movies About Gladiators', 'Mumblecore', 'Music', 'Musical', 'Musical Drama', 'Musical comedy', 'Mystery', 'Mythological Fantasy', 'Natural disaster', 'Natural horror films', 'Nature', 'Neo-noir', 'Neorealism', 'New Hollywood', 'New Queer Cinema', 'News', 'Ninja movie', 'Northern', 'Nuclear warfare', 'Operetta', 'Outlaw', 'Outlaw biker film', 'Parkour in popular culture', 'Parody', 'Patriotic film', 'Period Horror', 'Period piece', 'Pinku eiga', 'Plague', 'Point of view shot', 'Political Documetary', 'Political cinema', 'Political drama', 'Political satire', 'Political thriller', 'Pornographic movie', 'Pornography', 'Pre-Code', 'Prison', 'Prison escape', 'Prison film', 'Private military company', 'Propaganda film', 'Psycho-biddy', 'Psychological horror', 'Psychological thriller', 'Punk rock', 'Race movie', 'Reboot', 'Religious Film', 'Remake', 'Revenge', 'Revisionist Fairy Tale', 'Revisionist Western', 'Road movie', 'Road-Horror', 'Roadshow theatrical release', 'Roadshow/Carny', 'Rockumentary', 'Romance Film', 'Romantic comedy', 'Romantic drama', 'Romantic fantasy', 'Romantic thriller', 'Samurai cinema', 'Satire', 'School story', 'Sci Fi Pictures original films', 'Sci-Fi Adventure', 'Sci-Fi Horror', 'Sci-Fi Thriller', 'Science Fiction', 'Science fiction Western', 'Screwball comedy', 'Sex comedy', 'Sexploitation', 'Short Film', 'Silent film', 'Silhouette animation', 'Singing cowboy', 'Slapstick', 'Slasher', 'Slice of life story', 'Social issues', 'Social problem film', 'Softcore Porn', 'Space opera', 'Space western', 'Spaghetti Western', 'Splatter film', 'Sponsored film', 'Sports', 'Spy', 'Stand-up comedy', 'Star vehicle', 'Statutory rape', 'Steampunk', 'Stoner film', 'Stop motion', 'Superhero', 'Superhero movie', 'Supermarionation', 'Supernatural', 'Surrealism', 'Suspense', 'Swashbuckler films', 'Sword and Sandal', 'Sword and sorcery', 'Sword and sorcery films', 'Tamil cinema', 'Teen', 'Television movie', 'The Netherlands in World War II', 'Therimin music', 'Thriller', 'Time travel', 'Tokusatsu', 'Tollywood', 'Tragedy', 'Tragicomedy', 'Travel', 'Vampire movies', 'War effort', 'War film', 'Werewolf fiction', 'Western', 'Whodunit', 'Women in prison films', 'Workplace Comedy', 'World History', 'World cinema', 'Wuxia', 'Z movie', 'Zombie Film']

In [141]:
def make_prediction_pic(img):
    img = image.img_to_array(img)
    img = img/255
    proba = model.predict(img.reshape(1,400,400,3))
    ans = [(x, posters_genres[i]) for i, x in enumerate(proba[0])]
    ans = [x for x in ans if x[0] > 0.1]
    return ans

In [31]:
stop_words = set(open('prep/english').read().split("\n"))

In [114]:
def clean_text(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    text = text.lower() 
    
    return text

def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

def make_prediction_text(q):
    q = clean_text(q)
    q = remove_stopwords(q)
    q_vec = tfidf_vectorizer.transform([q])
    q_pred = clf.predict_proba(q_vec)
    return q_pred

In [148]:
pic = image.load_img('tests/2.jpg',target_size=(400,400,3))
ans = make_prediction_pic(pic)
print(ans)

[(0.16285333, 'Action'), (0.12171763, 'Adventure'), (0.46344805, 'Comedy'), (0.13839604, 'Crime'), (0.49206528, 'Drama'), (0.21319404, 'Romance')]


In [143]:
text = str(open('tests/1.txt').read())
raw_data = make_prediction_text(text)[0]

In [144]:
genres_result = [(x, plot_text_genres[i]) for i, x in enumerate(raw_data)]
real_genres = [x for x in genres_result if x[0] > 0.1]
real_genres.sort()
print(real_genres[::-1])

[(0.2572795364189539, 'Action'), (0.237487952438309, 'Drama'), (0.21364623624936038, 'Thriller'), (0.1441893888209462, 'Adventure'), (0.12121677722758692, 'Short Film'), (0.10103452247662455, 'Science Fiction')]
